# Этап 9: N:M Sparsity — Аппаратное ускорение разреженности

В отличие от случайного прунинга (Step 7), который сложно ускорить на обычном GPU, **N:M Sparsity** (например, 2:4) поддерживается аппаратно в ядрах Tensor Core (архитектура Ampere и новее).

### Идея метода:
В каждых 4 последовательных весах мы оставляем ровно 2 ненулевых. 
*   **Результат**: Мы гарантированно экономим 50% весов.
*   **Профит**: GPU может умножать такие матрицы почти в 2 раза быстрее.

In [ ]:
import torch
import torch.nn as nn
import copy
from src.model import GPTLanguageModel, device, estimate_loss

model = GPTLanguageModel().to(device)
model.load_state_dict(torch.load('model_ckpt.pt', map_location=device))
model.eval()

def apply_2_4_sparsity(layer):
    w = layer.weight.data
    B, C = w.shape
    
    # Разбиваем веса на группы по 4
    # Для простоты примера предположим, что C кратно 4
    w_reshaped = w.view(B, -1, 4)
    
    # Находим индексы 2-х самых маленьких по модулю весов в каждой группе
    _, indices = torch.topk(w_reshaped.abs(), k=2, dim=-1, largest=False)
    
    # Зануляем их
    mask = torch.ones_like(w_reshaped)
    mask.scatter_(dim=-1, index=indices, value=0)
    
    layer.weight.data = (w_reshaped * mask).view(B, C)
    return mask

# Применим к проекции первого блока
target_layer = model.blocks[0].sa.proj
apply_2_4_sparsity(target_layer)

def check_layer_sparsity(layer):
    total = layer.weight.numel()
    zeros = (layer.weight == 0).sum().item()
    print(f"Разреженность слоя: {100 * zeros / total:.1f}%")

check_layer_sparsity(target_layer)

## 2. SparseGPT: Умный выбор весов для зануления

Метод **SparseGPT** (упомянутый в обзоре) использует информацию о градиентах и обратной матрице Гессиана, чтобы занулять веса, которые меньше всего влияют на Loss, соблюдая паттерн 2:4.